# Create a Google Search Agent

In this tutorial, we will be using `council` to build a search agent that can leverage both Google Search and Google News to answer a user question about Microsoft.

Code from this notebook will be used to build the Financial Analyst Agent in `4_financial_analyst_agent`, but this notebook can also be executed to create a standalone search agent.

## Important Prerequisities

API keys for using Google Search are not provided. You will be asked to insert your own `GOOGLE_API_KEY` and `GOOGLE_SEARCH_ENGINE_ID` keys as environment variables.

To create a `GOOGLE_API_KEY`:

- Navigate to the [Google Cloud Console](https://console.cloud.google.com/) and log in with your Google account.
- Click the project drop-down and select or create the project for which you want to add an API key.
- Using the search bar, find the "APIs & Services" page
- Navigate to "Credentials" on the left-hand side
- Click the "Create credentials" drop-down and select "API key".

To create a `GOOGLE_SEARCH_ENGINE_ID`: 

- Navigate to Google's [Programmable Search Engine](https://programmablesearchengine.google.com/about/) page. Make sure you're logged in with the same Google account used to create your `GOOGLE_API_KEY`.
- Click the "Get Started" button
- Fill out the form to create a new search engine
- Find the Search engine ID 

## Install the council-ai library

In [ ]:
!pip install council-ai

## Import the required modules

In [ ]:
import json
import os

from typing import List
from string import Template

from council.skills import SkillBase, LLMSkill, PromptToMessages
from council.skills.google import GoogleSearchSkill, GoogleNewsSkill
from council.runners import Parallel
from council.contexts import SkillContext, ChatMessage, Budget
from council.filters import BasicFilter
from council.llm import OpenAILLM, LLMMessage
from council.prompt import PromptBuilder
from council.chains import Chain
from council.agents import Agent
from council.contexts import AgentContext
from council.controllers import BasicController
from council.evaluators import BasicEvaluator

import dotenv
dotenv.load_dotenv()

print(f"OPENAI_API_KEY set: {os.getenv('OPENAI_API_KEY', None) is not None}")
print(f"GOOGLE_SEARCH_ENGINE_ID set: {os.getenv('GOOGLE_SEARCH_ENGINE_ID', None) is not None}")
print(f"GOOGLE_API_KEY set: {os.getenv('GOOGLE_API_KEY', None) is not None}")

## Specifying constants used in the notebook

In [ ]:
COMPANY_NAME = "Microsoft"
COMPANY_TICKER = "MSFT"

## Initialize search skills
The [GoogleSearchSkill](https://github.com/chain-ml/council/blob/main/council/skills/google/google_search_skill.py) and [GoogleNewsSkill](https://github.com/chain-ml/council/blob/main/council/skills/google/google_news_skill.py) are available directly in council. We would like this agent to leverage search results from both Google Search and Google News, so we create another skill called `GoogleAggregator` to aggregate the results from both. This skill will search the context for the messages created by the `GoogleSearchSkill` and `GoogleNewsSkill`, extract their results, and combine the title and snippet from each search result to form paragraphs of text as context to an LLM.

More specifically, we can access the information for the current execution of the chain by calling the `current` method on the `context`. This will return the `ChainHistory` that contains all the messages in the chain, such as those from the user, an agent or from a skill itself. of We can then search the messages in the current chain execution to find those from the `GoogleSearchSkill` and `GoogleNewsSkill`. In council, every skill has a name, and the skill names for Search and News are `gsearch` and `gnews`, respectively. We use the `last_message_from_skill` method to find the search result.

In [ ]:
# Initializing Google News and Google Search skills
google_search_skill = GoogleSearchSkill()
google_news_skill = GoogleNewsSkill()


# Google aggregator skill
class GoogleAggregatorSkill(SkillBase):
    """Skill to aggregate results from Google Search and Google News"""

    def __init__(
        self,
    ):
        super().__init__(name="google_aggregator")

    def execute(self, context: SkillContext) -> ChatMessage:
        gsearch_results = (
            json.loads(context.current.last_message_from_skill("gsearch").data)
            if context.current.last_message_from_skill("gsearch").is_ok
            else []
        )
        gnews_results = (
            json.loads(context.current.last_message_from_skill("gnews").data)
            if context.current.last_message_from_skill("gnews").is_ok
            else []
        )
        search_results = gsearch_results + gnews_results

        context = ""
        for result in search_results:
            text = result.get("title", "") + " " + result.get("snippet", "") + "\n\n"
            context += text

        return self.build_success_message(context)

# Initializing Google aggregation skill
google_aggregator_skill = GoogleAggregatorSkill()

## Create the LLMSkill
The `LLMSkill` is a skill provided by council that can make a call and return a response from an LLM model. 
We will inject the context created from the retrieved documents by the `GoogleAggregatorSkill` into the `LLMSkill`.

In [ ]:
# LLM Skill
llm = OpenAILLM.from_env(model='gpt-3.5-turbo')

SYSTEM_MESSAGE = "You are a research analyst whose job it is to answer user questions about $company with the provided context."

PROMPT = """Use the following pieces of context to answer the query.
If the answer is not provided in the context, do not make up an answer. Instead, respond that you do not know.

CONTEXT:
{{chain_history.last_message}}
END CONTEXT.

QUERY:
{{chat_history.user.last_message}}
END QUERY.

YOUR ANSWER:
"""

def build_context_messages(context: SkillContext) -> List[LLMMessage]:
    """Context messages function for LLMSkill"""
    context_message_prompt = PromptToMessages(prompt_builder=PromptBuilder(PROMPT))
    return context_message_prompt.to_user_message(context)


llm_skill = LLMSkill(
    llm=llm,
    system_prompt=Template(SYSTEM_MESSAGE).substitute(company=COMPANY_NAME),
    context_messages=build_context_messages,
)

## Create the chain
Since the `GoogleSearchSkill` and `GoogleNewsSkill` can be run independently of each other, we will run the skills in parallel for the chain to execute faster, before aggregating them together in the `GoogleAggregatorSkill`.

council provides easy-to-use functionality to run skills in parallel within a chain. We simply wrap the search and news skill in the `Parallel` class.

In [ ]:
search_chain = Chain(
    name="search_chain",
    description=f"Information about {COMPANY_NAME} ({COMPANY_TICKER}) using a Google search",
    runners=[
        Parallel(google_search_skill, google_news_skill),
        google_aggregator_skill,
        llm_skill
    ],
)

## Create the search agent

The agent will use the search chain we created and the `BasicController` and `BasicEvaluator`, which will select our single chain and return the response to the user.

Notebooks `3_controller` and `4_financial_analyst_agent` will demonstrate more complicated uses of the controller and evaluator.

In [ ]:
agent = Agent(controller=BasicController(chains=[search_chain]), evaluator=BasicEvaluator(), filter=BasicFilter())

## Interact with the agent

In [ ]:
run_context = AgentContext.from_user_message("What is the financial performance of Microsoft?", budget=Budget(60))
result = agent.execute(run_context)
print(result.best_message.message)

In the [next part](./3_controller.ipynb), we will create a custom controller for our financial analyst. 